# VQE y problemas de optimización

In [2]:
%pip install qiskit qiskit-algorithms qiskit-optimization qiskit-aer


[notice] A new release of pip is available: 25.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Algoritmos Cuánticos
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals

# Componentes de Circuito y Primitivas (Versión Qiskit 1.0+)
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import StatevectorSampler

# --- 1. Definición del Problema (Datos de Entrada) ---
print("--- 1. INICIALIZANDO PROBLEMA UFLP ---")
num_bodegas = 2
num_clientes = 2

# Costos: Bodega 0 barata (10), Bodega 1 cara (20)
costos_instalacion = [10, 20] 

# Costos transporte:
# Bodega 0 -> Cliente 0 (1), Cliente 1 (5)
# Bodega 1 -> Cliente 0 (5), Cliente 1 (1)
costos_transporte = [
    [1, 5], 
    [5, 1]   
]
print(f"Instancia: {num_bodegas} bodegas, {num_clientes} clientes.")

--- 1. INICIALIZANDO PROBLEMA UFLP ---
Instancia: 2 bodegas, 2 clientes.


In [4]:
# --- 2. Modelado Matemático (QuadraticProgram) ---
qp = QuadraticProgram()

# Variables
for i in range(num_bodegas):
    qp.binary_var(name=f"x_{i}") # Bodegas
for i in range(num_bodegas):
    for j in range(num_clientes):
        qp.binary_var(name=f"y_{i}_{j}") # Enlaces

# Función Objetivo
linear_terms = {}
for i in range(num_bodegas):
    linear_terms[f"x_{i}"] = costos_instalacion[i]
for i in range(num_bodegas):
    for j in range(num_clientes):
        linear_terms[f"y_{i}_{j}"] = costos_transporte[i][j]
qp.minimize(linear=linear_terms)

# Restricciones
# 1. No asignar a bodega cerrada (y_ij <= x_i)
for i in range(num_bodegas):
    for j in range(num_clientes):
        qp.linear_constraint(linear={f"y_{i}_{j}": 1, f"x_{i}": -1}, sense="LE", rhs=0, name=f"link_{i}_{j}")

# 2. Cada cliente atendido por EXACTAMENTE una bodega (Sum(y_ij) == 1)
for j in range(num_clientes):
    linear_dict = {f"y_{i}_{j}": 1 for i in range(num_bodegas)}
    qp.linear_constraint(linear=linear_dict, sense="EQ", rhs=1, name=f"assign_{j}")

print("\nModelo clásico (primeras líneas):")
print(str(qp.prettyprint())[:300] + "...\n")


Modelo clásico (primeras líneas):
Problem name: 

Minimize
  10*x_0 + 20*x_1 + y_0_0 + 5*y_0_1 + 5*y_1_0 + y_1_1

Subject to
  Linear constraints (6)
    -x_0 + y_0_0 <= 0  'link_0_0'
    -x_0 + y_0_1 <= 0  'link_0_1'
    -x_1 + y_1_0 <= 0  'link_1_0'
    -x_1 + y_1_1 <= 0  'link_1_1'
    y_0_0 + y_1_0 == 1  'assign_0'
    y_0_1 + y...



In [5]:
# --- 3. Conversión a Hamiltoniano ---
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
op, offset = qubo.to_ising()

print(f"Número de Qubits necesarios: {op.num_qubits}")

Número de Qubits necesarios: 6


In [ ]:
# --- 4. Configuración y Ejecución del VQE con RESTARTS ---
print("\n--- EJECUTANDO VQE (Buscando el Mínimo Global) ---")

# Configuración fija
ansatz = EfficientSU2(num_qubits=op.num_qubits, reps=2, entanglement='linear')
optimizer = COBYLA(maxiter=500) # Aumentamos iteraciones
sampler = StatevectorSampler() # Sin semilla fija aquí, la variaremos en el bucle

min_costo_encontrado = float('inf')
mejor_solucion = None

# Intentaremos 10 veces con semillas diferentes
num_intentos = 10 

for i in range(num_intentos):
    print(f"Intento #{i+1}...", end=" ")
    
    # Cambiamos la semilla para cada intento
    semilla_actual = 100 + i
    algorithm_globals.random_seed = semilla_actual
    sampler = StatevectorSampler(seed=semilla_actual) # Semilla al sampler
    
    # Creamos VQE con la nueva semilla
    # Inicializamos puntos aleatorios diferentes para el optimizador
    initial_point = np.random.random(ansatz.num_parameters)
    
    vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=optimizer, initial_point=initial_point)
    vqe_solver = MinimumEigenOptimizer(vqe)
    
    try:
        result = vqe_solver.solve(qp)
        print(f"Costo: {result.fval}")
        
        # Guardamos si es el mejor hasta ahora
        if result.fval < min_costo_encontrado:
            min_costo_encontrado = result.fval
            mejor_solucion = result
            
    except Exception as e:
        print(f"Falló: {e}")

# --- 5. Análisis del MEJOR Resultado ---
print("\n--- 🏆 MEJOR RESULTADO ENCONTRADO ---")
if mejor_solucion:
    print(f"Estado óptimo (x, y): {mejor_solucion.x}")
    print(f"Costo Mínimo: {mejor_solucion.fval}")

    variables_activas = [var.name for var, val in zip(qp.variables, mejor_solucion.x) if val == 1]
    print(f"Decisión: {variables_activas}")

    # Validación Lógica
    bodegas = [v for v in variables_activas if 'x_' in v]
    if 'x_0' in bodegas and 'x_1' not in bodegas:
        print("✅ ¡ÉXITO! El algoritmo eligió la bodega barata (x_0).")
    elif 'x_1' in bodegas:
        print("⚠️ El algoritmo sigue prefiriendo la bodega cara. Intenta aumentar 'reps' en el Ansatz.")
else:
    print("No se encontraron soluciones válidas.")


--- EJECUTANDO VQE (Buscando el Mínimo Global) ---
Intento #1... 

C:\Users\yamca\AppData\Local\Temp\ipykernel_17116\2978949834.py:5: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  ansatz = EfficientSU2(num_qubits=op.num_qubits, reps=2, entanglement='linear')


Costo: 26.0
Intento #2... Costo: 16.0
Intento #3... Costo: 26.0
Intento #4... Costo: 16.0
Intento #5... Costo: 16.0
Intento #6... Costo: 16.0
Intento #7... Costo: 32.0
Intento #8... Costo: 16.0
Intento #9... Costo: 26.0
Intento #10... Costo: 16.0

--- 🏆 MEJOR RESULTADO ENCONTRADO ---
Estado óptimo (x, y): [1. 0. 1. 1. 0. 0.]
Costo Mínimo: 16.0
Decisión: ['x_0', 'y_0_0', 'y_0_1']
✅ ¡ÉXITO! El algoritmo eligió la bodega barata (x_0).


In [ ]:
# --- 1. Definición del Problema (Escenario: Zonas A y B) ---
print("--- 1. INICIALIZANDO CASO COMPLEJO (3 Bodegas, 4 Clientes) ---")
num_bodegas = 3
num_clientes = 4

# Costos de Inversión (k_i)
# Bodega 0 y 1 son baratas (15), Bodega 2 es cara (40)
costos_instalacion = [15, 15, 40]

# Costos de Transporte (c_ij)
# Matriz de 3 filas (bodegas) x 4 columnas (clientes)
# Clientes 0 y 1 están en "Zona Norte". Clientes 2 y 3 en "Zona Sur".
costos_transporte = [
    # C0, C1, C2, C3
    [1,  1,  10, 10],  # Bodega 0 (Norte): Barato para C0,C1. Caro para C2,C3
    [10, 10, 1,  1],   # Bodega 1 (Sur): Caro para C0,C1. Barato para C2,C3
    [5,  5,  5,  5]    # Bodega 2 (Central): Costo medio para todos
]

print(f"Instancia creada con {num_bodegas} bodegas y {num_clientes} clientes.")
print("Reto: ¿Conviene abrir la central cara o dos baratas distribuidas?")

# --- 2. Modelado Matemático ---
qp = QuadraticProgram()

# Variables Binarias
for i in range(num_bodegas):
    qp.binary_var(name=f"x_{i}") # x_0, x_1, x_2
for i in range(num_bodegas):
    for j in range(num_clientes):
        qp.binary_var(name=f"y_{i}_{j}") # Enlaces

# Función Objetivo
linear_terms = {}
for i in range(num_bodegas):
    linear_terms[f"x_{i}"] = costos_instalacion[i]
for i in range(num_bodegas):
    for j in range(num_clientes):
        linear_terms[f"y_{i}_{j}"] = costos_transporte[i][j]
qp.minimize(linear=linear_terms)

# Restricciones
# 1. Capacidad (y_ij <= x_i)
for i in range(num_bodegas):
    for j in range(num_clientes):
        qp.linear_constraint(linear={f"y_{i}_{j}": 1, f"x_{i}": -1}, sense="LE", rhs=0, name=f"link_{i}_{j}")

# 2. Asignación Única (Sum(y_ij) == 1 para cada cliente)
for j in range(num_clientes):
    linear_dict = {f"y_{i}_{j}": 1 for i in range(num_bodegas)}
    qp.linear_constraint(linear=linear_dict, sense="EQ", rhs=1, name=f"assign_{j}")

# --- 3. Conversión a Hamiltoniano ---
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
op, offset = qubo.to_ising()
print(f"\nComplejidad del Circuito: {op.num_qubits} Qubits requeridos.")

# --- 4. Ejecución VQE con Multi-Start ---
print("\n--- EJECUTANDO VQE (Buscando Mínimo Global) ---")
# Como el problema es más grande, usamos un Ansatz con menos profundidad (reps=1)
# para que el optimizador tenga menos parámetros que ajustar y converja mejor.
ansatz = EfficientSU2(num_qubits=op.num_qubits, reps=1, entanglement='linear')
optimizer = COBYLA(maxiter=1000) # Más iteraciones necesarias para 15 variables

min_costo = float('inf')
mejor_solucion = None

# Haremos varios intentos porque el espacio de búsqueda es grande (2^15 = 32,768 opciones)
num_intentos = 5 

for i in range(num_intentos):
    semilla = 50 + i * 10
    algorithm_globals.random_seed = semilla
    
    # Puntos iniciales aleatorios ayudan a explorar diferentes valles de energía
    initial_point = np.random.random(ansatz.num_parameters)
    
    sampler = StatevectorSampler(seed=semilla)
    vqe = SamplingVQE(sampler=sampler, ansatz=ansatz, optimizer=optimizer, initial_point=initial_point)
    vqe_solver = MinimumEigenOptimizer(vqe)
    
    try:
        print(f"Intento {i+1}/{num_intentos}...", end=" ")
        result = vqe_solver.solve(qp)
        print(f"Costo hallado: {result.fval}")
        
        if result.fval < min_costo:
            min_costo = result.fval
            mejor_solucion = result
    except Exception as e:
        print(f"Error: {e}")

# --- 5. Análisis e Interpretación ---
print("\n--- 🏆 RESULTADO ÓPTIMO FINAL ---")
if mejor_solucion:
    print(f"Costo Mínimo Global: {mejor_solucion.fval}")
    
    # Decodificación Inteligente
    vars_activas = [var.name for var, val in zip(qp.variables, mejor_solucion.x) if val == 1]
    bodegas_abiertas = sorted([v for v in vars_activas if 'x_' in v])
    enlaces = [v for v in vars_activas if 'y_' in v]
    
    print(f"\nBodegas a Construir: {bodegas_abiertas}")
    print(f"Distribución de Clientes ({len(enlaces)} asignados):")
    for b in bodegas_abiertas:
        # Filtramos qué clientes van a esta bodega
        idx_b = b.split('_')[1]
        clientes_b = [y.split('_')[2] for y in enlaces if y.split('_')[1] == idx_b]
        print(f"  -> {b}: Atiende a Clientes {clientes_b}")
        
    # Verificación de lógica de costos
    if len(bodegas_abiertas) == 2 and 'x_0' in vars_activas and 'x_1' in vars_activas:
        print("\n✅ ANÁLISIS: El algoritmo detectó correctamente que es mejor invertir")
        print("   en dos bodegas pequeñas (Costo 30) para minimizar el transporte,")
        print("   en lugar de una sola bodega central (Costo 40).")
    elif len(bodegas_abiertas) == 1 and 'x_2' in vars_activas:
        print("\n⚠️ ANÁLISIS: El algoritmo prefirió la bodega central. Es una solución válida")
        print("   pero quizás sub-óptima (Costo ~60 vs Costo 34 ideal).")
else:
    print("No convergió.")

--- 1. INICIALIZANDO CASO COMPLEJO (3 Bodegas, 4 Clientes) ---
Instancia creada con 3 bodegas y 4 clientes.
Reto: ¿Conviene abrir la central cara o dos baratas distribuidas?

Complejidad del Circuito: 15 Qubits requeridos.

--- EJECUTANDO VQE (Buscando Mínimo Global) ---
Intento 1/5... 

C:\Users\yamca\AppData\Local\Temp\ipykernel_17116\1407347425.py:63: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  ansatz = EfficientSU2(num_qubits=op.num_qubits, reps=1, entanglement='linear')


Costo hallado: 60.0
Intento 2/5... Costo hallado: 72.0
Intento 3/5... Costo hallado: 72.0
Intento 4/5... Costo hallado: 37.0
Intento 5/5... Costo hallado: 60.0

--- 🏆 RESULTADO ÓPTIMO FINAL ---
Costo Mínimo Global: 37.0

Bodegas a Construir: ['x_1']
Distribución de Clientes (4 asignados):
  -> x_1: Atiende a Clientes ['0', '1', '2', '3']
